# Simple GeoMesa Query

In [1]:
from pyspark.sql import SparkSession
import geomesa_pyspark

# Resource JARs for PySpark
jars=['/opt/geomesa/dist/spark/geomesa-hbase-spark-runtime.jar',
      '/usr/share/aws/emr/emrfs/lib/emrfs-hadoop-assembly-2.20.0.jar']
   
# Spark Configuration Parameters
conf = geomesa_pyspark.configure(
    jars=jars,
    packages=['geomesa_pyspark','pytz'],
    spark_home='/usr/lib/spark/').\
    setAppName('MyTestApp') 
conf.set('spark.jars', ','.join(jars))
conf.setAll([
    ('spark.executor.memory', '1g'), 
    ('spark.executor.cores', '2'), 
    ('spark.worker.instances', '2')])

# Initialize Spark Session
spark = ( SparkSession
    .builder
    .config(conf=conf)
    .enableHiveSupport()
    .getOrCreate()
)

# GeoMesa Database connection parameters
params = {
    "hbase.catalog": "catalogSmall"
}
feature = "gdelt"

# Retrieve Dataframe 
df = ( spark
    .read
    .format("geomesa")
    .options(**params)
    .option("geomesa.feature", feature)
    .load()
)

# Create SQL Temp View and print DataFrame Schema
df.createOrReplaceTempView("data")
df.printSchema()

In [2]:
spark.sql("""
select * from data
""").show()

In [4]:
spark.sql("""
select * from data where eventCode = '051'
""").show()